# Titanic data set

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/IBLM/blob/main/examples/iblmodel/iblmodel_titanic.ipynb)

In [1]:
%%capture
!pip install iblm --upgrade

In [2]:
import pandas as pd
from langchain.llms import OpenAI
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import seaborn as sns

from iblm import IBLModel

import os
#os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

### Preparing data sets

In [60]:
df = sns.load_dataset('titanic')

sample = 40
sample_num = int(sample/2)
seed = 3656

df['age'].fillna(df['age'].median(), inplace=True)
df['embarked'].fillna(df['embarked'].mode()[0], inplace=True)

df = pd.get_dummies(df, columns=['sex'])
df = pd.get_dummies(df, columns=['embarked'])
df = pd.get_dummies(df, columns=['alive'])
df = pd.get_dummies(df, columns=['alone'])
df = pd.get_dummies(df, columns=['adult_male'])
df = pd.get_dummies(df, columns=['who'])
df = pd.get_dummies(df, columns=['class'])
df = pd.get_dummies(df, columns=['deck'])
df = pd.get_dummies(df, columns=['embark_town'])
df = df.replace({True: 1, False: 0})

cols = list(df.columns)
cols.remove('survived')
cols.append('survived')
df = df[cols]

df_1 = df[df['survived'] == 1].sample(n = sample_num, random_state = seed)
df_0 = df[df['survived'] == 0].sample(n = sample_num, random_state = seed)

df_train = pd.DataFrame()
df_len = len(df_1)
for i in range(df_len):
    df1 = pd.DataFrame([df_1.iloc[i]])
    df0 = pd.DataFrame([df_0.iloc[i]])
    df_train = pd.concat([df_train, df1, df0])

df_train['survived'] = df_train['survived'].astype(int)
df_test = df.drop(df_train.index)
df_train.head()

,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S,...,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton,survived
431,3.0,28.00,1.0,0.0,16.1000,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
361,2.0,29.00,1.0,0.0,27.7208,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
305,1.0,0.92,1.0,2.0,151.5500,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
87,3.0,28.00,0.0,0.0,8.0500,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
781,1.0,17.00,1.0,0.0,57.0000,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [61]:
x_train = df_train.drop('survived', axis=1)
y_train = df_train['survived']

### Training

In [62]:
llm_model = OpenAI(temperature=0, model_name = 'gpt-4-0613')

params = {
    'columns_name': True,
    'objective': 'classification',
    }

iblm = IBLModel(llm_model = llm_model, params=params)

In [63]:
#file_path = '/content/'

#model = iblm.fit(x_train, y_train, model_name = 'titanic', file_path=file_path)
model = iblm.fit(x_train, y_train)

In [64]:
# Code Model
print(model)

import numpy as np

def predict(x):
    df = x.copy()
    
    output = []
    for index, row in df.iterrows():
        # Do not change the code before this point.
        # Please describe the process required to make the prediction below.
        
        # Initialize probability to 0.5
        prob = 0.5
        
        # If the passenger is female, increase the probability
        if row['sex_female'] == 1.0:
            prob += 0.2
            
        # If the passenger is in first class, increase the probability
        if row['class_First'] == 1.0:
            prob += 0.2
            
        # If the passenger is a child, increase the probability
        if row['who_child'] == 1.0:
            prob += 0.1
            
        # If the passenger embarked from Cherbourg, increase the probability
        if row['embark_town_Cherbourg'] == 1.0:
            prob += 0.05
            
        # If the passenger is alone, decrease the probability
        if row['alone_True'] == 1.0:


### Prediction

In [65]:
x_test = df_test.drop('survived', axis=1)
y_test = df_test['survived']

In [66]:
y_proba = iblm.predict(x_test)
y_pred = (y_proba > 0.5).astype(int)

In [67]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

# Precision
precision = precision_score(y_test, y_pred)
print(f'Precision: {precision}')

# Recall
recall = recall_score(y_test, y_pred)
print(f'Recall: {recall}')

# F1 score
f1 = f1_score(y_test, y_pred)
print(f'F1 score: {f1}')

# ROC-AUC (you need prediction probabilities for this, not just class predictions)
# Here we just reuse y_pred for simplicity
roc_auc = roc_auc_score(y_test, y_proba)
print(f'ROC-AUC: {roc_auc}')

Accuracy: 0.8049353701527615
Precision: 0.8451327433628318
Recall: 0.593167701863354
F1 score: 0.6970802919708029
ROC-AUC: 0.8468867780530474
